2025 - Employment Permit by County

In [27]:
from datetime import datetime

current_year = datetime.now().year

year = current_year

# Reading the raw data in the folder on my personal computer
import pandas as pd
import os

df = pd.read_excel(f"G:/My Drive/ESTUDOS DATA SCIENCE/ie-employment-permit/data/raw_data/{year}/permits-by-county-{year}.xlsx",
                header=1,
                skiprows=[2]
)
print(df)

           Unnamed: 0  Issued  Refused  Withdrawn
0   No County Entered     NaN      NaN     3985.0
1              Antrim     NaN      1.0        NaN
2              Carlow   122.0     13.0       66.0
3               Cavan   301.0     33.0      221.0
4               Clare   336.0     88.0      230.0
5                Cork  1867.0    199.0      798.0
6             Donegal   309.0     45.0      150.0
7                Down     1.0      NaN        NaN
8              Dublin  9277.0    897.0     3249.0
9              Galway   665.0     83.0      329.0
10              Kerry   355.0     26.0      157.0
11            Kildare   833.0    124.0      394.0
12           Kilkenny   378.0     34.0      126.0
13              Laois   139.0     52.0       71.0
14            Leitrim    49.0      3.0       33.0
15           Limerick  1023.0    113.0      465.0
16           Longford    90.0     15.0       26.0
17              Louth   296.0     30.0      107.0
18               Mayo   223.0     29.0      123.0


In [28]:
# As the structure of the columns were modified after 2020, I had to rename automatically the first one by "County"
df.rename(columns={"Unnamed: 0": "County"}, inplace=True)

In [29]:
# Creating a new column because of the new structure where the year is on top of the table
df["Year"] = year

# Sorting the year column to be the first one 
df = df[["Year"] + [col for col in df.columns if col != "Year"]]

In [30]:
# Reading counties/provinces reference
counties_df = pd.read_excel("G:/My Drive/ESTUDOS DATA SCIENCE/ie-employment-permit/data/counties_and_provincies.xlsx")

# Creating the Obs column
def get_obs(county):
    if county == "No County Entered":
        return county
    # If the County isnt on the file, its Northern Ireland
    match = counties_df[counties_df["County"] == county]
    if match.empty:
        return "Northern Ireland"
    else:
        return "" 

df["Obs"] = df["County"].apply(get_obs)

In [31]:
# As this part of project is focused only in the counties, I had to group all missed places by "Others"

# df["Obs"] == "" → Northern Ireland
# df["Obs"] != "" → Others (North Ireland ou No County Entered)


# Filtering rows that needs to be "Others"
df_others = df[df["Obs"] != ""].copy()
df_others["County"] = "Others"
df_others_grouped = df_others.groupby(["Year", "County"], as_index=False)[["Issued", "Refused", "Withdrawn"]].sum(min_count=1)

# Republic of Ireland's rows (Obs == "")
df_main = df[df["Obs"] == ""].copy()
df = pd.concat([df_main, df_others_grouped], ignore_index=True)

# Removing Obs column
df.drop(columns=["Obs"], inplace=True)


In [32]:
# Creates the folder if it doesnt exist
output_dir = f"G:/My Drive/ESTUDOS DATA SCIENCE/ie-employment-permit/data/{year}"
os.makedirs(output_dir, exist_ok=True)

# Save as csv
csv_path = os.path.join(output_dir, f"permits-issued-by-county-{year}.csv")
df.to_csv(csv_path, index=False)

print(df)

    Year     County  Issued  Refused  Withdrawn
0   2025     Carlow   122.0     13.0       66.0
1   2025      Cavan   301.0     33.0      221.0
2   2025      Clare   336.0     88.0      230.0
3   2025       Cork  1867.0    199.0      798.0
4   2025    Donegal   309.0     45.0      150.0
5   2025     Dublin  9277.0    897.0     3249.0
6   2025     Galway   665.0     83.0      329.0
7   2025      Kerry   355.0     26.0      157.0
8   2025    Kildare   833.0    124.0      394.0
9   2025   Kilkenny   378.0     34.0      126.0
10  2025      Laois   139.0     52.0       71.0
11  2025    Leitrim    49.0      3.0       33.0
12  2025   Limerick  1023.0    113.0      465.0
13  2025   Longford    90.0     15.0       26.0
14  2025      Louth   296.0     30.0      107.0
15  2025       Mayo   223.0     29.0      123.0
16  2025      Meath   831.0    142.0      358.0
17  2025   Monaghan   326.0     27.0       91.0
18  2025     Offaly   234.0     21.0       69.0
19  2025  Roscommon   126.0     18.0    